In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain, LLMChain
import os
import json

In [2]:
llm = ChatGoogleGenerativeAI(
    model='gemini-pro',
    google_api_key=os.getenv('GEMINI_API_KEY'),
    convert_system_message_to_human=True
)

In [3]:
json_question_output_format = {
    "questions": [
        {
            "question": "What is the capital of France?",
            "options": {
                "A": "Berlin",
                "B": "Madrid",
                "C": "Paris",
                "D": "Rome"
            },
            "correct_answer": "C"
        },
        {
            "question": "Which gas do plants primarily absorb during photosynthesis?",
            "options": {
                "A": "Oxygen",
                "B": "Carbon dioxide",
                "C": "Nitrogen",
                "D": "Hydrogen"
            },
            "correct_answer": "B"
        }
    ]
}

In [123]:
question_generation_prompt = PromptTemplate(

    input_variables=['number_of_questions', 'topic',
                     'difficulty_level', 'json_question_output_format'],

    template='''
    You are an Smart Assistant AI that generates multiple-choice questions (MCQs) for a given topic.
    There will be three difficulty level for the questions. for example : easy, medium, hard. Each question should have four answer options, one of which is correct. Make sure the questions are not repeated.
    
    Based on the input, generate {number_of_questions} multiple-choice questions on the topic "{topic}". The Diificulty Level of the question is {difficulty_level}

Must follow Instructions:
- don't put output json format into double quotation marks like "..."
- use '`' symbol in the content of questions, options and answers instead of "'".
- strictly follow given example output format.
- Make sure each question has four answer options, clearly labeled as A, B, C, and D.
- Ensure that only one answer is correct.
- Generate varied questions that cover different aspects of the topic, but stay relevant.
- Return the questions in the following format:

Example output:

{json_question_output_format}

    '''
)

In [124]:
question_generator_chain = LLMChain(
    llm=llm, prompt=question_generation_prompt, verbose=True)

In [140]:
generated_question = question_generator_chain.run({'number_of_questions': 25, 'topic': 'python',
                                                   'difficulty_level': 'medium', "json_question_output_format": json_question_output_format})



> Entering new LLMChain chain...
Prompt after formatting:

    You are an Smart Assistant AI that generates multiple-choice questions (MCQs) for a given topic.
    There will be three difficulty level for the questions. for example : easy, medium, hard. Each question should have four answer options, one of which is correct. Make sure the questions are not repeated.
    
    Based on the input, generate 25 multiple-choice questions on the topic "python". The Diificulty Level of the question is medium

Must follow Instructions:
- don't put output json format into double quotation marks like "..."
- use '`' symbol in the content of questions, options and answers instead of "'".
- strictly follow given example output format.
- Make sure each question has four answer options, clearly labeled as A, B, C, and D.
- Ensure that only one answer is correct.
- Generate varied questions that cover different aspects of the topic, but stay relevant.
- Return the questions in the following format:



In [141]:
generated_question

'{\'questions\': [{\'question\': `In Python, what is the output of the following code:\nmy_list = [1, 2, 3, 4, 5]\nmy_new_list = my_list[::2]\nprint(my_new_list)`, \'options\': {\'A\': \'[1, 2, 3, 4, 5]\', \'B\': \'[1, 3, 5]\', \'C\': \'[2, 4]\', \'D\': \'[]\'}, \'correct_answer\': \'B\'}, {\'question\': `Which of the following is a valid syntax to create a dictionary in Python?`, \'options\': {\'A\': \'my_dict = {"key1": "value1", "key2": "value2"}\', \'B\': \'my_dict = {key1: "value1", key2: "value2"}\', \'C\': \'my_dict = dict({"key1": "value1", "key2": "value2"})\', \'D\': \'All of the above\'}, \'correct_answer\': \'D\'}, {\'question\': `What is the purpose of the \'break\' statement in Python?`, \'options\': {\'A\': \'To terminate the execution of a loop\', \'B\': \'To skip the current iteration of a loop\', \'C\': \'To exit a function\', \'D\': \'To handle exceptions\'}, \'correct_answer\': \'A\'}, {\'question\': `Which of the following is NOT a data type in Python?`, \'options\

In [142]:
generated_question = generated_question.replace("'", '"')
print(generated_question)

{"questions": [{"question": `In Python, what is the output of the following code:
my_list = [1, 2, 3, 4, 5]
my_new_list = my_list[::2]
print(my_new_list)`, "options": {"A": "[1, 2, 3, 4, 5]", "B": "[1, 3, 5]", "C": "[2, 4]", "D": "[]"}, "correct_answer": "B"}, {"question": `Which of the following is a valid syntax to create a dictionary in Python?`, "options": {"A": "my_dict = {"key1": "value1", "key2": "value2"}", "B": "my_dict = {key1: "value1", key2: "value2"}", "C": "my_dict = dict({"key1": "value1", "key2": "value2"})", "D": "All of the above"}, "correct_answer": "D"}, {"question": `What is the purpose of the "break" statement in Python?`, "options": {"A": "To terminate the execution of a loop", "B": "To skip the current iteration of a loop", "C": "To exit a function", "D": "To handle exceptions"}, "correct_answer": "A"}, {"question": `Which of the following is NOT a data type in Python?`, "options": {"A": "Integer", "B": "Float", "C": "String", "D": "Array"}, "correct_answer": "D

In [143]:
ans = json.loads(generated_question)

JSONDecodeError: Expecting value: line 1 column 29 (char 28)

In [139]:
# Open a file in write mode
with open('questions_and_answers.txt', 'w') as file:
    for i in range(50):
        try:
            # Write the formatted content to the file
            file.write(f"""
            Question {i+1}: {ans['questions'][i]['question']}
            Options : 
                A. {ans['questions'][i]['options']['A']}
                B. {ans['questions'][i]['options']['B']}
                C. {ans['questions'][i]['options']['C']}
                D. {ans['questions'][i]['options']['D']}
            Correct Answer : {ans['questions'][i]['options'][ans['questions'][i]['correct_answer']]}
            \n
            """)
        except IndexError:
            continue
        except KeyError as e:
            # Handle the case where a key is missing in the data
            file.write(f"Question {i+1}: Error - Missing key {e}\n\n")
        except Exception as e:
            # Handle any other exceptions
            file.write(f"Question {i+1}: Error - {str(e)}\n\n")